In [10]:
%matplotlib inline
import os
from osgeo import gdal
import rasterio
import geopandas as gpd
import rasterio.mask
import torch
import sys
sys.path.append(r"C:\Users\52551\Documents\Trabajo\codigos\Geolocalizacion\geoloc2\Detecciondeterrenos")
from codigos import Generar_txt
###path de yolo dentro de computadora
os.chdir(r"C:\Users\52551\Documents\Trabajo\Equipo_Agua\Geo\Data\yolov7")
from detect_Alberto_v4 import *

In [11]:
Modelo=modelo(weights="Modelos/best_Hector_2.pt",device="cpu") ## device="0" para gpu

YOLOR  v0.1-122-g3b41c2c torch 1.13.1+cpu CPU



Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse


Model Summary: 314 layers, 36487166 parameters, 6194944 gradients, 103.2 GFLOPS


 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 



## Parametros del raster

In [12]:
raster=r"G:\.shortcut-targets-by-id\19j98C-PJkrk3gkgU7V03dLiJ4Gn-hnNe\geoshaps\Ixtapaluca\Ixta_3.tif"

In [13]:
gdal_interpeter = gdal.Open(raster)
width = gdal_interpeter.RasterXSize
height = gdal_interpeter.RasterYSize
coordenadas_gdal = gdal_interpeter.GetGeoTransform()
minx = coordenadas_gdal[0]
miny = coordenadas_gdal[3] + width*coordenadas_gdal[4] + height*coordenadas_gdal[5] 
maxx = coordenadas_gdal[0] + width*coordenadas_gdal[1] + height*coordenadas_gdal[2]
maxy = coordenadas_gdal[3]
src_raster_path = raster
src=rasterio.open(src_raster_path)
H,W=src.shape
src.crs,H,W,minx,maxx,miny,maxy,"W_metros",maxx-minx,"H_metros",maxy-miny

CPLE_AppDefined in PROJ: proj_create_from_database: C:\Users\52551\anaconda3\envs\yolov7\Library\share\proj\proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.
CPLE_AppDefined in The definition of projected CRS EPSG:3857 got from GeoTIFF keys is not the same as the one from the EPSG registry, which may cause issues during reprojection operations. Set GTIFF_SRS_SOURCE configuration option to EPSG to use official parameters (overriding the ones from GeoTIFF keys), or to GEOKEYS to use custom values from GeoTIFF keys and drop the EPSG code.


(CRS.from_wkt('LOCAL_CS["WGS 84 / Pseudo-Mercator",UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'),
 38569,
 71403,
 -11015486.2918,
 -10979784.736,
 2184152.0008,
 2203436.3419,
 'W_metros',
 35701.555800000206,
 'H_metros',
 19284.341099999845)

In [14]:
dim=int(np.ceil(map_d(minx+120,minx,maxx,0,W)))
alto=int(np.floor(H/dim))
ancho=int(np.floor(W/dim))
alto,ancho,alto*ancho
nombres=[]
imshow=False
result=pd.DataFrame()
casas=[]
terreno=[]
angulosget=[]
alto,ancho,alto*ancho,dim

(160, 297, 47520, 240)

## Generar imagenes mosaico

In [16]:
# Path donde se guardan images
path_mosaico_salida=r"G:\Mi unidad\Equipo_Agua\Geo\Data\Prueba\ixtapaluca\Images1/"
generar_imagenes=int(input("Generar imagenes: 1 si, 0 no "))
if generar_imagenes==1:
    generar_imagenes_sinrotar=int(input("Generar imagenes sin rotar: 1 si, 0 no "))
    generar_imagenes_rotadas=int(input("Generar imagenes rotadas: 1 si, 0 no "))
    with tqdm.tqdm(total=alto*ancho) as pbar:
        for j in range(0,ancho):#ancho
            for i in (range(alto)):#alto
                label=raster.replace("\\","/").split("/")[-1][:-4]+"_"
                nameimg=label.lower()+str(i)+"_"+str(j)
                cuadro=[]
                for k in range(2):
                    for l in range(2):
                        cuadro.append((minx+(maxx-minx)/ancho*(j+k),
                                       maxy-(maxy-miny)/alto*(i+l),
                                       0.0))
                cuadro=[cuadro[0],cuadro[1],cuadro[3],cuadro[2],cuadro[0]]
                shapes=[{"type":'Polygon','coordinates':[cuadro]}]
                array, out_transform = rasterio.mask.mask(src, shapes, crop=True)
                four_images=[array[2],array[1],array[0]]
                imagen_n = np.stack(four_images, axis=-1)
                if generar_imagenes_sinrotar==1:
                    cv2.imwrite(path_mosaico_salida+nameimg+".png",imagen_n)
                if generar_imagenes_rotadas==1:
                    angulo_1,imagen_ro=correct_orientation(imagen_n,dim)
                    cv2.imwrite(path_mosaico_salida+nameimg+"_"+str(angulo_1)+".png",imagen_ro)
                pbar.update(1)

  0%|          | 120/47520 [06:43<28:47:11,  2.19s/it]

## Generar prediccion sobre imagen con salida txt

In [ ]:
vector=Modelo.detect(opt_source=path_mosaico_salida,opt_conf_thres=0.08)
Generar_txt(vector,path_mosaico_salida+"labels")

## Generar prediccion sobre raster(municipio) con salida en shp

In [ ]:
imshow=False
with tqdm.tqdm(total=alto*ancho) as pbar:
    for j in range(ancho):#ancho
        for i in (range(alto)):#alto
            label=raster.replace("\\","/").split("/")[-1][:-4]+"_"
            nameimg=label.lower()+str(i)+"_"+str(j)
            nombres.append(nameimg)
            cuadro=[]
            for k in range(2):
                for l in range(2):
                    cuadro.append((minx+(maxx-minx)/ancho*(j+k),
                                   maxy-(maxy-miny)/alto*(i+l),
                                   0.0))
            cuadro=[cuadro[0],cuadro[1],cuadro[3],cuadro[2],cuadro[0]]
            shapes=[{"type":'Polygon','coordinates':[cuadro]}]
            vector=[]
            array, out_transform = rasterio.mask.mask(src, shapes, crop=True)
            four_images=[array[2],array[1],array[0]]
            imagen_n = np.stack(four_images, axis=-1)
            angulo_f=0
            anterior=0
            actual=1
            vector1=[]
            angulo=0
            le=0
            angulo_1=correct_orientation(imagen_n,dim)
            for i in range(-2,2,1):
                angulo=angulo_1+i
                anterior=len(vector)
                image_ro=imagen_n.copy()
                M = cv2.getRotationMatrix2D((dim//2,dim//2), angulo, 1)
                image_ro = cv2.warpAffine(image_ro, M, (dim,dim))
                with torch.no_grad():
                    vector1=Modelo.detect(opt_source="cache1.png",opt_conf_thres=0.08,imagen_s=image_ro)
                actual=len(vector)
                if len(vector1)>le:
                    le=len(vector1)
                    vector=vector1
                    imagen_n1=image_ro
                    angulo_f=angulo        
            vector2=Modelo.detect(opt_source="cache1.png",opt_conf_thres=0.08,imagen_s=imagen_n)
            if len(vector)>len(vector2):
                imagen_n1=image_ro
                angulo_f=angulo
            else:
                le=len(vector2)
                vector=vector2
                imagen_n1=imagen_n
                angulo_f=0
            angulo=angulo_f
#                 if angulo!=0:
#                     imshow=True
            angulosget.append(angulo)
            proyecciones=shapes[0].get("coordinates")[0][:-1]
            df_cache=vector2xy(vector,dim=dim,nameimg=nameimg)
            
            for i in (range(len(df_cache))):
                if df_cache["Tipo"][i]=="casa":
                    casas.append(rotacion_detect(df_cache.loc[i,"start_point_100"], df_cache.loc[i,"end_point_100"],-angulo,proyecciones=proyecciones))
                else:
                    terreno.append(rotacion_detect(df_cache.loc[i,"start_point_100"], df_cache.loc[i,"end_point_100"],-angulo,proyecciones=proyecciones))
            if imshow:
                print(angulo)
                imshow_detect(df_cache,imagen_n1)
                imshow=False
                
            pbar.update(1)


In [ ]:
gdf_casas=gpd.GeoDataFrame(geometry=casas,crs=src.crs)
gdf_casas.set_crs=src.crs
gdf_casas["area"]=gdf_casas.area
gdf_casas.to_file(r"C:/Users/ASUS/salida/shape/new_casas.shp")
gdf_terreno=gpd.GeoDataFrame(geometry=terreno,crs=src.crs)
gdf_terreno.set_crs=src.crs
gdf_terreno.to_file(r"C:/Users/ASUS/salida/shape/new_terreno.shp")